In [ ]:
! yes y | %reset
! yes y |conda install -c conda-forge gensim 

In [ ]:
#%reset

In [2]:
import os, sys
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
import re

In [3]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs2"
logdir = "{}/Staticrun-{}/".format(root_logdir,now)


In [4]:
def read_data(foldername):
    filenames=[]
    for fname in os.listdir(foldername):
        filenames.append(fname)
        
    return filenames

In [5]:
existing_files = read_data('/home/ubuntu/Static')
existingfilenames_df = pd.DataFrame(np.array(existing_files).reshape(len(existing_files),1), columns = ['apk'])
existingfilenames_df.head()

,apk
0,43cf70608f210b973a5439dcac952a54.txt
1,ACB70A1C7F00915FBAFA19B8FFC1E16528158B69614A19...
2,AD0D2A7E5728179689F603A57F124CA81C31676CE65DC7...
3,AACAA1647BEC55A3DFFD5088F8CCAB2B4261CD24AC9F82...
4,AE21B3E03926AD4F2EA4D7688DE2A2556B3589DE9BB603...


In [6]:
fileswithlabel = pd.read_csv("/home/ubuntu/labelledfileswith-txt.txt")
fileswithlabel.head()

,apk,label
0,AD12012BFD86BC73765A73277537B0E1F753379CD7CC9E...,0
1,AD120A3582AD38EC1833AB5ABB0ED243ADE70616F798E8...,0
2,AD1218C053BCA5096FFF94C28F1DCFAA8B60259FFA3382...,0
3,AD121D186298CD48BFBB0A28872DB6CAB4E96BE129856A...,0
4,AD122295A328DB33B30BEF8156FCA80DA20701CE566B2B...,0


In [7]:
fileslistwithlabel = existingfilenames_df.set_index('apk').join(fileswithlabel.set_index('apk'))
del existingfilenames_df

In [8]:

fileslistwithlabel.head()

,label
apk,
43cf70608f210b973a5439dcac952a54.txt,1
ACB70A1C7F00915FBAFA19B8FFC1E16528158B69614A19BFF49A401B8C1D7DC4.txt,0
AD0D2A7E5728179689F603A57F124CA81C31676CE65DC7DDB83BB005032A7008.txt,0
AACAA1647BEC55A3DFFD5088F8CCAB2B4261CD24AC9F827CECB32AB566045E63.txt,0
AE21B3E03926AD4F2EA4D7688DE2A2556B3589DE9BB603F20FB61F95D8E8C943.txt,0


In [9]:
fileslistwithlabel.reset_index(inplace=True)

In [10]:
#fileslistwithlabel.reset_index(drop=True,inplace=True)

In [11]:
len(fileslistwithlabel)

48051

In [12]:
del fileswithlabel
fileslistwithlabel.head()

,apk,label
0,43cf70608f210b973a5439dcac952a54.txt,1
1,ACB70A1C7F00915FBAFA19B8FFC1E16528158B69614A19...,0
2,AD0D2A7E5728179689F603A57F124CA81C31676CE65DC7...,0
3,AACAA1647BEC55A3DFFD5088F8CCAB2B4261CD24AC9F82...,0
4,AE21B3E03926AD4F2EA4D7688DE2A2556B3589DE9BB603...,0


In [13]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)

In [14]:
for train_index,test_index in split.split(fileslistwithlabel,fileslistwithlabel['label']):
    #print(train_index[:10])
    strat_train_set = fileslistwithlabel.loc[train_index]
    strat_test_set = fileslistwithlabel.loc[test_index]

In [15]:
del fileslistwithlabel
strat_test_set['label'].value_counts() / len(strat_test_set)

0    0.751014
1    0.248986
Name: label, dtype: float64

In [16]:
len(strat_test_set)

9611

In [17]:
strat_train_set['label'].value_counts() / len(strat_train_set)

0    0.751041
1    0.248959
Name: label, dtype: float64

In [18]:
len(strat_train_set)

38440

In [19]:
#strat_train_set.set_index('apk')

In [20]:
strat_train_set.reset_index(drop=True,inplace=True)

In [21]:
strat_train_set.head()

,apk,label
0,AC6513BC89B7D6B058D192570D7411A8D99116F0533E91...,0
1,AD488D16BB126B851D4D8ACC32E6A5C3ACE981CEE4B3D4...,0
2,AC61E54052E6A21143EB05DA849FA76C358927ECE23AD6...,0
3,ABEAA41DAD82EB8E6BC728DAB1B3EDCD05E45CE1F166F4...,0
4,AB77B5F70884E9DFDB2454F46CDC32046D761003229EA6...,0


In [22]:
#!pip3 install --upgrade gensim --user

#### Also get the pretrained-wordtovec model


In [23]:
from gensim.models import KeyedVectors
model = KeyedVectors.load("/home/ubuntu/wordvectors.kv", mmap='r')
#model = gensim.models.Word2Vec.load('/home/ubuntu/model-all-52dim-skipgram')

In [24]:
model

In [25]:
model.wv.vocab

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{'Ljava/lang/String;.<init>:([BIILjava/lang/String;)V': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0a160>,
 'Ljava/lang/Object;.<init>:()V': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0a1d0>,
 'Ljava/lang/Object;.getClass:()Ljava/lang/Class;': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0ae10>,
 'Ljava/lang/Class;.getName:()Ljava/lang/String;': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0acf8>,
 'Ljava/lang/StringBuilder;.<init>:()V': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0ad30>,
 'Ljava/lang/String;.substring:(II)Ljava/lang/String;': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0ab70>,
 'Ljava/lang/StringBuilder;.append:(Ljava/lang/String;)Ljava/lang/StringBuilder;': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0ab38>,
 'Ljava/lang/Exception;.<init>:()V': <gensim.models.keyedvectors.Vocab at 0x7f3f07d0add8>,
 'Ljava/lang/Class;.getDeclaredMethod:(Ljava/lang/String;[Ljava/lang/Class;)Ljava/lang/reflect/Method;': <gensim.models.keyedvectors.Vocab at 0x7f3f07a93240>,

In [26]:
model.wv['Ljava/lang/String;.<init>:([BIILjava/lang/String;)V']

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


memmap([ 0.7027573 ,  0.2357014 ,  0.50791395,  0.5066296 , -0.07097847,
        -0.89561117,  0.4107526 ,  0.5397846 ,  0.29125744,  0.80521417,
         0.07104103, -0.20695022,  0.4575843 , -0.45108658, -0.05520138,
         0.82249326, -0.54668325,  0.58708084,  0.05244777, -0.9105996 ,
         0.33870474,  0.06001737, -0.10135366, -0.10188413,  0.44558525,
        -0.23854348, -0.18795718,  0.39265135, -0.03374079, -0.44190425,
        -0.5278169 , -0.13389234,  0.17642395, -0.8558    ,  0.28307894,
        -0.5883036 ,  0.44654626,  0.68218887, -0.6389782 ,  0.08484074,
         0.6812076 , -0.5540381 ,  1.2570051 ,  0.568323  , -0.19316052,
        -0.18834612,  0.16485605,  0.15295045,  0.2568722 , -0.40117458,
         0.19130266,  0.32208824], dtype=float32)

# #Now prepare the Train-data for K-fold cross validation

In [27]:
from sklearn.model_selection import StratifiedKFold

In [28]:
batch_size = 300

###Prepare the machine learning model here too, we will feed the data to this model using skfold after createing the model

In [29]:
# Training Parameters
learning_rate = 0.001
training_steps = 500
              #means there are 84 batches in an epoch with each batch having 150 apps 
total_epochs = 20
display_step = 200
# Network Parameters
num_input = 52 # MNIST data input (img shape: 28*28)
timesteps = 4000 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 2 # Binary classification
# tf Graph input
tf.reset_default_graph()
with tf.name_scope("placeholders"):
    xin = tf.placeholder("float", [None, timesteps, num_input],name="xin")
    Y = tf.placeholder("float", [None, num_classes],name="Y")
with tf.name_scope("weights"):
    weights = {    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))}
with tf.name_scope("bias"):
    biases = {    'out': tf.Variable(tf.random_normal([num_classes]))}
def RNN(xin, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    xin = tf.unstack(xin, timesteps, 1) #xin = tf.unstack(xin, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, xin, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [30]:
from tensorflow.contrib import rnn
with tf.name_scope("RNN"):
    logits = RNN(xin, weights, biases)
with tf.name_scope("Softmax"):
    prediction = tf.nn.softmax(logits)
# Define loss and optimizer
with tf.name_scope("Lossmeasure"):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
with tf.name_scope("Optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [31]:
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("evaluation"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
with tf.name_scope("Accuracy"):
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [32]:
loss_summary = tf.summary.scalar('LOSS',loss_op)
accuracy_summary = tf.summary.scalar('Accuracy',accuracy)

In [33]:
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
init = tf.global_variables_initializer()

In [38]:
batch_num_for_graph=0
def run_train(session,which_fold,X_train,y_train):
    print ("Start Training")
    session.run(init)
    global batch_num_for_graph
    merged = tf.summary.merge_all()
    epoch_accuracy=[]
    epoch_loss=[]
    for epoch in range (3):
        batch_accuracy=[]
        batch_loss=[]
        
        #total_batch = int(len(X_train)/batch_size)
        total_batch=2
        for i in range (total_batch):
            #now prepare data for each batch
            
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            #now we have the relevant filenames only
            batch_y = make_label(batch_y)
            sentences, _ = read_data("/home/ubuntu/Static/",batch_x)
            #print("read data")
            #nwo we need to convert sentences to word-embedded features and then we are ready to train
            batch_x = get_embedding(sentences)
            #print("got embedding")
            # Reshape data to get 28 seq of 28 elements
            batch_x = batch_x.reshape((batch_size,timesteps,num_input))
            # Run optimization op (backprop)
            loss,acc,summary,_ = session.run([loss_op,accuracy,merged,train_op], feed_dict={xin:batch_x,Y:batch_y})
            batch_accuracy.append(acc)
            batch_loss.append(loss)
            if  i==(batch_size-1) or (i==0 and which_fold==1) :
            
                # Calculate batch loss and accuracy
                #summary_str = loss_summary.eval(session=sess,feed_dict={xin: batch_x,Y: batch_y})
                #loss, acc,summary = session.run([loss_op,accuracy,merged],feed_dict={xin: batch_x,Y: batch_y})
                #file_writer.add_summary(summary,(epoch+which_fold+i)*batch_size)
                print("benign  = "+str(batch_y[:,0].sum()) + "  malicious = "+str(batch_y[:,1].sum()))
                print("Fold " + str(which_fold) + " Epoch "+str(epoch)+",Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))  
        epoch_accuracy.append(np.mean(batch_accuracy))
        epoch_loss.append(np.mean(batch_loss))
        summary = tf.Summary()
        summary.value.add(tag="Batch Accuracy", simple_value=np.mean(batch_accuracy))
        summary.value.add(tag="Batch Loss",simple_value=np.mean(batch_loss))
        file_writer.add_summary(summary, batch_num_for_graph)
        batch_num_for_graph+=1
    return np.mean(epoch_accuracy), np.mean(epoch_loss)
                # Create a new Summary object with your measure
                #summary = tf.Summary()
                #summary.value.add(tag="%s Batch Accuracy" %step, simple_value=batch_acc)
                #summary.value.add(tag="%s Batch Loss" %step, simple_value=batch_ls)
                # Add it to the Tensorboard summary writer
                # Make sure to specify a step parameter to get nice graphs over time
                #file_writer.add_summary(summary, count)
                #file_writer.add_summary(accuracy_summary_str,step)
    #             summary = tf_session.run(tf.summary.merge_all())
                #file_writer.add_summary(summary_str, step)
                #loss, acc = sess.run([loss_op, accuracy], feed_dict={xin: batch_x,Y: batch_y})
                

In [39]:
def run_validation_or_test(session,X_train,y_train):
    batch_accuracy=[]
    batch_loss=[]
    #total_batch = int(len(X_train)/batch_size)
    total_batch=2
    for i in range (total_batch):
        batch_x = X_train[i*batch_size:(i+1)*batch_size]
        batch_y = y_train[i*batch_size:(i+1)*batch_size]
        #now we have the relevant filenames only
        batch_y = make_label(batch_y)
        sentences, _ = read_data("/home/ubuntu/Static/",batch_x)
        #print("read data")
        #nwo we need to convert sentences to word-embedded features and then we are ready to train
        batch_x = get_embedding(sentences)
        #print("got embedding")
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size,timesteps,num_input))
        # Run optimization op (backprop)
        loss,acc = session.run([loss_op,accuracy],feed_dict={xin: batch_x,Y: batch_y})
        print("Accuracy-acc = " +str(acc) + "benign  = "+str(batch_y[:,0].sum()) + "  malicious = "+str(batch_y[:,1].sum()))
        batch_accuracy.append(acc)
        batch_loss.append(loss)
    return np.mean(batch_accuracy), np.mean(batch_loss)
    
        
        

In [40]:
def read_data(foldername,fileslist):
    sentences=[]
    numwords=[]
    for fname in fileslist:
        with open(os.path.join(foldername, fname)) as f:
            content = f.read().strip()
            content = re.sub(r"[\n#]","",content).split(" ")
            counter = len(content)
            numwords.append(counter)   #only consider the 4000 APIs# talk and decide later// part of tuning
        sentences.append(content[:4000])
        
    return sentences,numwords

def make_label(val):
    
    d = np.zeros((val.size,2))
    i = 0;
    for item in val:
        if item==0: #if benign
            d[i][0]=1.0
        else:
            d[i][1]=1.0
        i+=1
    return d
def get_embedding(sentences):
    d = np.zeros((len(sentences),4000,52))
    i=0
    for sentence in sentences:
        j=0
        for word in sentence:
            d[i][j]= model.wv[word]
            j+=1
        i+=1
    return d
            
    

In [41]:
def cross_validate(session,split_size=5):
    results=[]
    validation_num=0
    skf = StratifiedKFold(n_splits=split_size,shuffle=True,random_state=42)
    per_kminus1_fold_train_accuracy=[]
    per_kminus1_fold_train_loss=[]
    which_fold=1
    for train_index, test_index in skf.split(strat_train_set['apk'].tolist(),strat_train_set['label'].tolist()):
        train = strat_train_set.loc[train_index]
        print(len(train_index))
        print(len(test_index))
        test = strat_train_set.loc[test_index]
        X_train = train['apk']  #training data features; currently just filenames
        X_test = test['apk']   
        y_train = train['label']  #training data label
        y_test = test['label']
       
        #now run the train
        epoch_acc, epoch_loss = run_train(session,which_fold,X_train,y_train)
        print("trained a fold %s " %which_fold)
        per_kminus1_fold_train_accuracy.append(epoch_acc)
        per_kminus1_fold_train_loss.append(epoch_loss)
        which_fold+=1
        #now obtian validation testing data usign the filenames
        
        acc,loss = run_validation_or_test(session,X_test,y_test)
        print("Overall Accuracy -for the fold "+ str(acc)+" loss = "+str(loss))    
        summary = tf.Summary()
        summary.value.add(tag="Validation Accuracy", simple_value=acc)
        summary.value.add(tag="Validation loss", simple_value=loss)
       
        # Add it to the Tensorboard summary writer
        # Make sure to specify a step parameter to get nice graphs over time
        file_writer.add_summary(summary, validation_num)
        validation_num+=1
        results.append(acc)
    return results,per_kminus1_fold_train_accuracy,per_kminus1_fold_train_loss
        

In [42]:
with tf.Session() as session:
    result,per_fold_accuracy,per_fold_loss = cross_validate(session,5)
    print ("Cross validation result: %s" %result)
    X_test = strat_test_set['apk']
    y_test = strat_test_set['label']
#     sentences, _ = read_data("/home/ubuntu/Static/",X_test)
#     labels = make_label(y_test)
    #nwo we need to convert sentences to word-embedded features and then we are ready to TEST on validation
#     embedded_sentences = get_embedding(sentences)
    #now run the FINAL TEST
    acc,loss = run_validation_or_test(session,X_test,y_test)
    print("Overall Accuracy -for the fold "+ str(acc)+" loss = "+str(loss)) 
    #print ("Test Accuracy %f"% session.run(accuracy,feed_dict={xin: embedded_sentences,Y: labels}))

30752
7688
Start Training


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


benign  = 223.0  malicious = 77.0
Fold 1 Epoch 0,Minibatch Loss= 1.3233, Training Accuracy= 0.350
benign  = 223.0  malicious = 77.0
Fold 1 Epoch 1,Minibatch Loss= 1.0470, Training Accuracy= 0.460
benign  = 223.0  malicious = 77.0
Fold 1 Epoch 2,Minibatch Loss= 0.8939, Training Accuracy= 0.497
trained a fold 1 
Accuracy-acc = 0.49333334benign  = 222.0  malicious = 78.0
Accuracy-acc = 0.48333332benign  = 220.0  malicious = 80.0
Overall Accuracy -for the fold 0.48833334 loss = 0.86045444
30752
7688
Start Training
trained a fold 2 
Accuracy-acc = 0.42benign  = 224.0  malicious = 76.0
Accuracy-acc = 0.51benign  = 220.0  malicious = 80.0
Overall Accuracy -for the fold 0.46499997 loss = 0.7365248
30752
7688
Start Training
trained a fold 3 
Accuracy-acc = 0.76666665benign  = 235.0  malicious = 65.0
Accuracy-acc = 0.74666667benign  = 226.0  malicious = 74.0
Overall Accuracy -for the fold 0.75666666 loss = 0.5395248
30752
7688
Start Training
trained a fold 4 
Accuracy-acc = 0.27benign  = 245.0  

In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

In [ ]:
# for train_index, test_index in skf.split(strat_train_set['apk'].tolist(),strat_train_set['label'].tolist()):
#     train = strat_train_set.loc[train_index]
#     test = strat_train_set.loc[test_index]
#     X_train = train['apk']
#     X_test = test['apk']
#     y_train = train['label']
#     y_test = test['label']
#     #print(train[:10])
#     print(len(train))
#     print(len(X_train))
#     print(len(X_test))
    
    

In [ ]:
# filesnameonly = fileslistwithlabel['apk'].tolist()
# labelsonly = fileslistwithlabel['label'].tolist()

In [ ]:
# def simple_zip(batch_size=10):
#     dx = tf.data.Dataset.from_tensor_slices(filesnameonly)
#     dy = tf.data.Dataset.from_tensor_slices(labelsonly)
#     dcomb = tf.data.Dataset.zip((dx,dy)).repeat().batch(batch_size)
#     iterator = dcomb.make_initializable_iterator()
#     next_element = iterator.get_next()
#     with tf.Session() as sess:
#         sess.run(iterator.initializer)
#         for i in range(2):
#             val = sess.run(next_element)
#             print(val)

In [ ]:
# simple_zip(2)